In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12303,2024-02-25,Eua Nba,15:00,Philadelphia 76ers,Milwaukee Bucks,2.69,1.39,228.5,1.84,1.96,4.5,1.98,1.29,CUUrWvs6,0.371747,0.719424,0.543478,0.510204,0.091172,0.2,0.2,NaN,NaN,263.692,139.611718,0.529450,187.770,67.409866,0.359002,224.130,214.442,279.76,282.24,0.0,0.0,0.0,0.0,0.450607,0.044659,0.298412,-2.31,-0.462,-3.658009,0.572210,0.5,-0.072210,-0.95,-0.190,-2.052632,0.645793,0.4,-0.245793
12304,2024-02-25,Eua Nba,17:30,Phoenix Suns,Los Angeles Lakers,1.48,2.41,238.5,1.85,1.96,-5.5,2.02,3.39,O4YnVbdC,0.675676,0.414938,0.540541,0.510204,0.090613,0.6,0.8,NaN,NaN,195.198,47.052690,0.241051,232.696,79.591942,0.342043,168.434,315.858,125.28,147.56,0.0,0.0,0.0,0.0,0.338102,0.040830,0.358128,1.97,0.394,1.218274,0.713189,0.9,0.186811,0.98,0.196,7.193878,0.423666,0.4,-0.023666
12305,2024-02-25,Eua Nba,19:00,Indiana Pacers,Dallas Mavericks,1.88,1.78,250.5,1.85,1.95,-2.5,1.97,2.13,IRXjUIBI,0.531915,0.561798,0.540541,0.512821,0.093713,0.6,0.2,NaN,NaN,158.346,32.357993,0.204350,190.692,64.220370,0.336775,240.404,215.756,151.51,207.40,0.0,0.0,0.0,0.0,0.038640,0.037216,0.055189,3.04,0.608,1.447368,0.566255,0.7,0.133745,0.27,0.054,14.444444,0.437703,0.5,0.062297
12306,2024-02-25,Eua Nba,21:00,Atlanta Hawks,Orlando Magic,1.83,2.04,228.5,1.82,1.98,-2.5,1.99,2.15,YZFwhM3P,0.546448,0.490196,0.549451,0.505051,0.036644,0.8,0.6,NaN,NaN,251.438,166.588295,0.662542,173.698,31.614737,0.182010,248.376,238.288,202.86,214.70,0.0,0.0,0.0,0.0,0.076740,0.059546,0.054656,0.41,0.082,10.121951,0.577639,0.6,0.022361,0.70,0.140,7.428571,0.459011,0.4,-0.059011
12307,2024-02-25,Eua Nba,21:00,Golden State Warriors,Denver Nuggets,1.98,1.71,231.5,1.80,2.01,1.5,1.97,1.76,r95DoKtg,0.505051,0.584795,0.555556,0.497512,0.089846,0.4,0.4,NaN,NaN,250.660,111.117429,0.443299,170.682,44.689534,0.261829,211.982,190.342,109.61,181.45,0.0,0.0,0.0,0.0,0.103479,0.077949,0.079621,-0.43,-0.086,-11.395349,0.610300,0.6,-0.010300,-3.39,-0.678,-1.047198,0.571255,0.4,-0.171255
12308,2024-02-25,Eua Nba,21:00,Houston Rockets,Oklahoma City Thunder,2.41,1.48,233.5,1.81,2.00,4.5,2.05,1.26,zq25mtCt,0.414938,0.675676,0.552486,0.500000,0.090613,0.4,0.4,NaN,NaN,245.442,51.333969,0.209149,198.082,25.646948,0.129476,223.208,183.306,251.94,193.14,0.0,0.0,0.0,0.0,0.338102,0.070525,0.337531,1.38,0.276,5.108696,0.505594,0.6,0.094406,-1.63,-0.326,-1.472393,0.583159,0.5,-0.083159
12309,2024-02-25,Europa Campeonato Mundial,07:00,Azerbaijan,Kosovo,2.20,1.64,149.5,1.87,1.87,1.5,2.02,1.50,ncd81Ljp,0.454545,0.609756,0.534759,0.534759,0.064302,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.206239,0.000000,0.208918,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12310,2024-02-25,Europa Campeonato Mundial,13:00,Luxemburgo,Noruega,2.23,1.62,150.5,1.84,1.90,1.5,2.05,1.48,nHOgDIkd,0.448430,0.617284,0.543478,0.526316,0.065714,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.224070,0.022688,0.228357,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12311,2024-02-25,Mundo Americup,20:10,Colombia,Venezuela,2.29,1.58,146.5,1.86,1.84,-2.5,1.96,1.76,MXebsDp1,0.436681,0.632911,0.537634,0.543478,0.069593,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.259455,0.007644,0.076033,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
12312,2024-02-25,Mundo Americup,22:10,Bahamas,Porto Rico,1.6

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:30,Eua Nba,Phoenix Suns,Los Angeles Lakers,238.5,1.85,1.0,Over
1,19:00,Eua Nba,Indiana Pacers,Dallas Mavericks,250.5,1.85,1.0,Over
2,21:00,Eua Nba,Atlanta Hawks,Orlando Magic,228.5,1.82,1.0,Over
3,22:10,Mundo Americup,Bahamas,Porto Rico,164.5,1.87,1.0,Over
4,12:30,Áustria Liga Zweite,Dornbirn Lions,Salzburg,154.5,1.85,1.0,Over
5,00:00,Eua Ncaa,UCLA,USC,133.5,1.91,1.0,Over
6,01:59,Eua Ncaa,Hawaii,Long Beach State,150.5,1.80,1.0,Over
7,12:30,Portugal Proliga,Braga,CA Queluz,157.5,1.80,1.0,Over
8,08:00,Portugal Lfb Feminina,Galitos F,Natacao F,125.5,1.85,1.0,Over
